# `nba_api` demo

Let's call `cumeplayerstats` endpoint and parse the results

In [2]:
from nba_api.stats.endpoints import commonplayerinfo, cumestatsplayer

player_info = commonplayerinfo.CommonPlayerInfo(player_id=2544, timeout=30)

In [5]:
import pandas as pd

df_info = player_info.common_player_info.get_data_frame()
df_info

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,2544,LeBron,James,LeBron James,"James, LeBron",L. James,lebron-james,1984-12-30T00:00:00,St. Vincent-St. Mary HS (OH),USA,...,lebron_james,2003,2022,N,Y,Y,2003,1,1,Y


In [6]:
df_info.columns

Index(['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST',
       'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG',
       'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT',
       'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS',
       'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME',
       'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE',
       'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG',
       'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG'],
      dtype='object')

In [7]:
df_seasons = player_info.available_seasons.get_data_frame()
df_seasons

,SEASON_ID
0,12003
1,22003
2,12004
3,22004
4,32004
...,...
67,42020
68,52020
69,12021
70,22021


In [9]:
# Need `game_ids` for our cumulative search:
try:
    season_avg = cumestatsplayer.CumeStatsPlayer(player_id=2544)
except TypeError as e:
    print(e)
    print("Need list of game_ids")

__init__() missing 1 required positional argument: 'game_ids'
Need list of game_ids


[According to docs](https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/cumestatsplayer.md), also need:

* league_id
* season
* season_type_all_star: `^(Regular Season)|(Pre Season)|(Playoffs)|(All Star)$`

Example valid url: 

> `https://stats.nba.com/stats/cumestatsplayer?GameIDs=0021700807&LeagueID=00&PlayerID=2544&Season=2019-20&SeasonType=Regular+Season`

So league_id, season, season_type_all_star we can hardcode:

* league_id = `0`
* season = `2021-2022`
* season_type_all_star = `"Regular Season"`

But we'll need a way to dynamically fetch game_ids, given the player_id and season

In [43]:
from nba_api.stats.endpoints import playergamelog

gamelogs = playergamelog.PlayerGameLog(
    player_id=2544,
    season="2020-21",
    season_type_all_star="Playoffs",
)
gamelogs.get_data_frames()


[  SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  \
 0     42020       2544  0042000156  JUN 03, 2021  LAL vs. PHX  L   41   11   
 1     42020       2544  0042000155  JUN 01, 2021    LAL @ PHX  L   32    9   
 2     42020       2544  0042000154  MAY 30, 2021  LAL vs. PHX  L   38   10   
 3     42020       2544  0042000153  MAY 27, 2021  LAL vs. PHX  W   38    9   
 4     42020       2544  0042000152  MAY 25, 2021    LAL @ PHX  W   39    9   
 5     42020       2544  0042000151  MAY 23, 2021    LAL @ PHX  L   36    6   
 
    FGA  FG_PCT  ...  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  \
 0   26   0.423  ...     7    9    7    2    2    3   1   29          -3   
 1   19   0.474  ...     4    5    7    0    0    3   1   24         -24   
 2   21   0.476  ...    10   12    6    1    0    6   1   25           6   
 3   19   0.474  ...     5    6    9    1    0    7   0   21          15   
 4   16   0.563  ...     4    4    9    2    0    1   3   23     

In [29]:
df_game_ids = gamelogs.get_data_frames()[0]['Game_ID']
df_game_ids

0     0022101160
1     0022101119
2     0022101072
3     0022101061
4     0022101051
5     0022101039
6     0022101028
7     0022101019
8     0022101004
9     0022100985
10    0022100959
11    0022100945
12    0022100931
13    0022100919
14    0022100904
15    0022100884
16    0022100854
17    0022100829
18    0022100822
19    0022100799
20    0022100718
21    0022100700
22    0022100685
23    0022100680
24    0022100665
25    0022100649
26    0022100626
27    0022100606
28    0022100589
29    0022100566
30    0022100551
31    0022100538
32    0022100520
33    0022100511
34    0022100491
35    0022100487
36    0022100469
37    0022100448
38    0022100436
39    0022100420
40    0022100403
41    0022100389
42    0022100380
43    0022100365
44    0022100341
45    0022100304
46    0022100291
47    0022100268
48    0022100249
49    0022100232
50    0022100107
51    0022100093
52    0022100077
53    0022100040
54    0022100025
55    0022100002
Name: Game_ID, dtype: object

In [13]:
# in json format
import json
gamelogs_json = json.loads(gamelogs.get_json())
gamelogs_json.keys()

dict_keys(['resource', 'parameters', 'resultSets'])

In [26]:
gamelogs_json['resultSets'][0]['headers']

['SEASON_ID',
 'Player_ID',
 'Game_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'PLUS_MINUS',
 'VIDEO_AVAILABLE']

In [15]:
normal = json.loads(gamelogs.get_normalized_json())
normal.keys()

dict_keys(['PlayerGameLog'])

In [18]:
normal['PlayerGameLog'][0]

{'SEASON_ID': '22021',
 'Player_ID': 2544,
 'Game_ID': '0022101160',
 'GAME_DATE': 'APR 01, 2022',
 'MATCHUP': 'LAL vs. NOP',
 'WL': 'L',
 'MIN': 40,
 'FGM': 13,
 'FGA': 23,
 'FG_PCT': 0.565,
 'FG3M': 3,
 'FG3A': 8,
 'FG3_PCT': 0.375,
 'FTM': 9,
 'FTA': 12,
 'FT_PCT': 0.75,
 'OREB': 0,
 'DREB': 8,
 'REB': 8,
 'AST': 4,
 'STL': 1,
 'BLK': 0,
 'TOV': 6,
 'PF': 3,
 'PTS': 38,
 'PLUS_MINUS': -2,
 'VIDEO_AVAILABLE': 1}

In [34]:
game_ids_list = [normal['PlayerGameLog'][i]['Game_ID'] for i in range(len(normal['PlayerGameLog']))]
game_ids_list

['0022101160',
 '0022101119',
 '0022101072',
 '0022101061',
 '0022101051',
 '0022101039',
 '0022101028',
 '0022101019',
 '0022101004',
 '0022100985',
 '0022100959',
 '0022100945',
 '0022100931',
 '0022100919',
 '0022100904',
 '0022100884',
 '0022100854',
 '0022100829',
 '0022100822',
 '0022100799',
 '0022100718',
 '0022100700',
 '0022100685',
 '0022100680',
 '0022100665',
 '0022100649',
 '0022100626',
 '0022100606',
 '0022100589',
 '0022100566',
 '0022100551',
 '0022100538',
 '0022100520',
 '0022100511',
 '0022100491',
 '0022100487',
 '0022100469',
 '0022100448',
 '0022100436',
 '0022100420',
 '0022100403',
 '0022100389',
 '0022100380',
 '0022100365',
 '0022100341',
 '0022100304',
 '0022100291',
 '0022100268',
 '0022100249',
 '0022100232',
 '0022100107',
 '0022100093',
 '0022100077',
 '0022100040',
 '0022100025',
 '0022100002']

Seems easier to just get it from a dataframe and slice it by `["Game_ID"]`. But `json` avoids loading `pandas`. Using `get_normalized_json` flattens the response, so we can separate the IDs by

```py
normal = json.loads(response.get_normalized_json())
game_ids_list = [normal['PlayerGameLog'][i]['Game_ID'] for i in range(len(normal['PlayerGameLog']))]
```

In terms of finding player_id, we can use the static library:

In [35]:
from nba_api.stats.static import players

# Find players by full name.
players.find_players_by_full_name('james')

# Find players by first name.
players.find_players_by_first_name('lebron')

# Find players by last name.
players.find_players_by_last_name('^(james|love)$')

# Get all players.
all = players.get_players()

active = players.get_active_players()
inactive = players.get_inactive_players()

In [36]:
active[0]

{'id': 1630173,
 'full_name': 'Precious Achiuwa',
 'first_name': 'Precious',
 'last_name': 'Achiuwa',
 'is_active': True}

In [37]:
fred = players.find_players_by_full_name('vanvleet')
fred

[{'id': 1627832,
  'full_name': 'Fred VanVleet',
  'first_name': 'Fred',
  'last_name': 'VanVleet',
  'is_active': True}]

In [38]:
james = [players.find_players_by_full_name('james')
james

[{'id': 202341,
  'full_name': 'James Anderson',
  'first_name': 'James',
  'last_name': 'Anderson',
  'is_active': False},
 {'id': 200788,
  'full_name': 'James Augustine',
  'first_name': 'James',
  'last_name': 'Augustine',
  'is_active': False},
 {'id': 76085,
  'full_name': 'James Bailey',
  'first_name': 'James',
  'last_name': 'Bailey',
  'is_active': False},
 {'id': 1806,
  'full_name': 'James Blackwell',
  'first_name': 'James',
  'last_name': 'Blackwell',
  'is_active': False},
 {'id': 1630547,
  'full_name': 'James Bouknight',
  'first_name': 'James',
  'last_name': 'Bouknight',
  'is_active': True},
 {'id': 1530,
  'full_name': 'James Collins',
  'first_name': 'James',
  'last_name': 'Collins',
  'is_active': False},
 {'id': 1526,
  'full_name': 'James Cotton',
  'first_name': 'James',
  'last_name': 'Cotton',
  'is_active': False},
 {'id': 201191,
  'full_name': 'JamesOn Curry',
  'first_name': 'JamesOn',
  'last_name': 'Curry',
  'is_active': False},
 {'id': 76524,
  'ful

In [44]:
def get_params(*params):
    for param in params:
        print(param)

get_params(1, 2, "3")

1
2
3


In [39]:
james_active = [player for player in james if player['is_active']]
james_active

[{'id': 1630547,
  'full_name': 'James Bouknight',
  'first_name': 'James',
  'last_name': 'Bouknight',
  'is_active': True},
 {'id': 201935,
  'full_name': 'James Harden',
  'first_name': 'James',
  'last_name': 'Harden',
  'is_active': True},
 {'id': 2544,
  'full_name': 'LeBron James',
  'first_name': 'LeBron',
  'last_name': 'James',
  'is_active': True},
 {'id': 201949,
  'full_name': 'James Johnson',
  'first_name': 'James',
  'last_name': 'Johnson',
  'is_active': True},
 {'id': 1630164,
  'full_name': 'James Wiseman',
  'first_name': 'James',
  'last_name': 'Wiseman',
  'is_active': True}]

In [47]:
def get_game_ids(*gamelogs: json) -> list:
    game_ids = []
    for season_types in gamelogs:
        log_json = json.loads(season_types)
        game_id_type = [log_json['PlayerGameLog'][i]['Game_ID'] \
                    for i in range(len(log_json['PlayerGameLog']))]
        game_ids.append(game_id_type)

    return game_ids
        

In [52]:
# get collection of game_id for each of the jameses

import time

james_active_ids = [player['id'] for player in james_active]

james_game_ids = {}
for player_id in james_active_ids:
    reg = playergamelog.PlayerGameLog(
        player_id=player_id,
        season="2021-22",
        season_type_all_star="Regular Season",
    ).get_normalized_json()
    post = playergamelog.PlayerGameLog(
        player_id=player_id,
        season="2021-22",
        season_type_all_star="Playoffs",
    ).get_normalized_json()
    game_ids = get_game_ids(reg, post)
    james_game_ids[player_id] = game_ids
    time.sleep(0.4)


In [65]:
lebron = 2544
shorts = james_game_ids[lebron][0][:5]

['0022101160', '0022101119', '0022101072', '0022101061', '0022101051']

In [70]:
try:
    lebron = 2544
    season_avg = cumestatsplayer.CumeStatsPlayer(
        player_id=lebron,
        game_ids=james_game_ids[lebron][0][:5],g
        season="2021-22",
        season_type_all_star="Regular Season",
    )
except TypeError as e:
    print(e)

In [71]:
season_avg.get_request_url()

'https://stats.nba.com/stats/cumestatsplayer?GameIDs=0022101160&GameIDs=0022101119&GameIDs=0022101072&GameIDs=0022101061&GameIDs=0022101051&LeagueID=00&PlayerID=2544&Season=2021-22&SeasonType=Regular+Season'

In [72]:
season_df = season_avg.game_by_game_stats.get_data_frame()
season_df

,DATE_EST,VISITOR_TEAM,HOME_TEAM,GP,GS,ACTUAL_MINUTES,ACTUAL_SECONDS,FG,FGA,FG_PCT,...,TOT_REB,AVG_TOT_REB,AST,PF,DQ,STL,TURNOVERS,BLK,PTS,AVG_PTS
0,04/01/2022,Pelicans,Lakers,1,1,39,31,13,23,0.565,...,8,8.0,4,3,0,1,6,0,38,38.0


In [63]:
james_game_ids[lebron][0]

['0022101160',
 '0022101119',
 '0022101072',
 '0022101061',
 '0022101051',
 '0022101039',
 '0022101028',
 '0022101019',
 '0022101004',
 '0022100985',
 '0022100959',
 '0022100945',
 '0022100931',
 '0022100919',
 '0022100904',
 '0022100884',
 '0022100854',
 '0022100829',
 '0022100822',
 '0022100799',
 '0022100718',
 '0022100700',
 '0022100685',
 '0022100680',
 '0022100665',
 '0022100649',
 '0022100626',
 '0022100606',
 '0022100589',
 '0022100566',
 '0022100551',
 '0022100538',
 '0022100520',
 '0022100511',
 '0022100491',
 '0022100487',
 '0022100469',
 '0022100448',
 '0022100436',
 '0022100420',
 '0022100403',
 '0022100389',
 '0022100380',
 '0022100365',
 '0022100341',
 '0022100304',
 '0022100291',
 '0022100268',
 '0022100249',
 '0022100232',
 '0022100107',
 '0022100093',
 '0022100077',
 '0022100040',
 '0022100025',
 '0022100002']